In [1]:
!pip install ipython ipywidgets
!pip install firebase-admin
!pip install firebase


  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import datetime, timedelta
import plotly.graph_objects as go
import nltk
import re
from collections import defaultdict
from nltk.chat.util import Chat, reflections
from firebase import firebase
from collections import Counter
import seaborn as sns
from ipywidgets import VBox, HBox



# Establish Firebase connection

In [3]:
# Firebase URL
firebase_url = 'https://cloud-tut6-2a99a-default-rtdb.firebaseio.com/'
FBconn = firebase.FirebaseApplication(firebase_url, None)

# Retrieve a json file from firebase

In [4]:
# Function to fetch data from Firebase based on file name
def fetch_data_by_file_name(file_name):
    result = FBconn.get('/JsonFiles', None)
    if result:
        for key, value in result.items():
            if value.get('file_name') == file_name:
                data= value.get('data')  # Return the associated data

                # Remove entries that do not contain 'Document'
                data = [entry for entry in data if 'Document' in entry]

                return data
    return None

def prep_data(file_name):
  data=fetch_data_by_file_name(file_name)

  df = pd.DataFrame(data)
  df['Time'] = pd.to_datetime(df['Time'])

  return data, df

data, df=prep_data('OnshapeJson0.json')
cur_file_name="OnshapeJson0.json"


# PageBase class

In [5]:
# Base class for pages
class PageBase:
    def __init__(self):
        self.container = widgets.VBox()
        self.create_widgets()
        self.hide()  # Hide content initially

    def create_widgets(self):
        raise NotImplementedError("Subclasses should implement create_widgets()")

    def show(self):
        self.container.layout.display = 'block'

    def hide(self):
        self.container.layout.display = 'none'




# Main Page class

In [6]:

class MainPage(PageBase):
    def __init__(self):
        super().__init__()
        self.year_dropdown = None
        self.project_dropdown = None
        self.employees = self.getEmployees()
        self.create_widgets()

    def create_widgets(self):
        self.content = widgets.HTML(value='<h1>Main Page Content</h1>')
        self.container = widgets.VBox()
        self.container.children = [self.content]

        self.basic_stats = widgets.HTML(value="")
        self.output_grid = [[widgets.Output(), widgets.Output()],
                            [widgets.Output(), widgets.Output()]]

        self.grid = widgets.GridBox(
            children=[output for row in self.output_grid for output in row],
            layout=widgets.Layout(
                grid_template_columns="repeat(2, 1fr)",
                grid_template_rows="repeat(2, auto)",
                grid_gap="20px"
            )
        )

        self.update_main_info()
        self.container.children = [self.content, self.basic_stats, self.grid]

        # Dropdown for selecting the year
        self.year_dropdown = widgets.Dropdown(
            options=df['Time'].dt.year.unique(),
            description='Year:',
            disabled=False,
        )

        # Dropdown for selecting the year for projects graph
        self.project_dropdown = widgets.Dropdown(
            options=df['Time'].dt.year.unique(),
            description='Year:',
            disabled=False,
        )

        # Function to update the plot when the dropdown value changes
        def on_year_change(change):
            with self.output_grid[1][0]:
                clear_output(wait=True)
                display(self.year_dropdown)
                self.GraphTotalAction(change['new'])

        # Function to update the projects plot when the dropdown value changes
        def on_project_year_change(change):
            with self.output_grid[1][1]:
                clear_output(wait=True)
                display(self.project_dropdown)
                self.GraphActionsPerProjects(change['new'])

        # Attach the event listener to the dropdown
        self.year_dropdown.observe(on_year_change, names='value')
        self.project_dropdown.observe(on_project_year_change, names='value')

        # Place dropdowns above respective graphs without initially plotting the graphs
        with self.output_grid[1][0]:
            clear_output(wait=True)
            display(self.year_dropdown)
            self.GraphTotalAction(2022)


        with self.output_grid[1][1]:
            clear_output(wait=True)
            display(self.project_dropdown)
            self.GraphActionsPerProjects(2022)


        with self.output_grid[0][0]:
            clear_output(wait=True)
            display(HTML(self.employees))

    def getEmployees(self):
        Employees = {}
        for emp in df['User'].unique():
            tmp = []
            tmp.append(len(df[df['User'] == emp]))
            tmp.append(len((df[df['User'] == emp])['Document'].unique()))
            Employees[emp] = tmp

        html_table = """
        <style>
            table {
                width: 100%;
                border-collapse: collapse;
                background-color: white;
            }
            th, td {
                padding: 12px;
                text-align: center;
                border-bottom: 1px solid #ddd;
                color: black;
            }
            th {
                background-color: #4CAF50;
                color: white;
            }
            tr:hover {
                background-color: #f5f5f5;
            }
            .gray-text {
                color: gray; /* Set the text color to gray */
            }
        </style>
        <table>
            <thead>
                <tr>
                    <th>Employee</th>
                    <th>Projects</th>
                </tr>
            </thead>
            <tbody>
        """

        # Add the employee names and actions
        for emp, value in Employees.items():
            html_table += f"""
                <tr>
                    <td>
                        <div>{emp}</div>
                        <div class="gray-text">{value[0]} Actions</div>
                    </td>
                    <td>
                        {value[1]}
                    </td>
                </tr>
            """

        html_table += """
            </tbody>
        </table>
        """
        return html_table

    def CalculateTotalActions(self, year):
        totalActionsPerMonths = [0] * 12
        for i in range(12):
            totalActionsPerMonths[i] = len(df[(df['Time'].dt.year == year) & (df['Time'].dt.month == (i + 1))])
        return totalActionsPerMonths

    def GraphTotalAction(self, year=None):
        if year is None:
            return
        thisYear = self.CalculateTotalActions(year)
        previousYear = self.CalculateTotalActions(year - 1)
        monthsName = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

        frame = pd.DataFrame({
            'Month': monthsName,
            str(year): thisYear,
            str(year - 1): previousYear
        })

        # Plot
        plt.figure(figsize=(12, 8))
        plt.plot(frame['Month'], frame[str(year)], label=str(year), color='blue', marker='o')
        plt.plot(frame['Month'], frame[str(year - 1)], label=str(year - 1), color='red', marker='o')
        plt.xlabel('Month')
        plt.ylabel('Total Actions')
        plt.title('Total Actions')
        plt.legend()
        plt.grid(True)
        plt.show()

    def GraphActionsPerProjects(self, year=None):
        if year is None:
            return
        projects = df['Document'].unique()
        actionsPerProject = []
        for i in projects:
            actionsPerProject.append(len(df[(df['Document'] == i) & (df['Time'].dt.year == year)]))

        frame = pd.DataFrame({
            'Document': projects,
            'Actions': actionsPerProject,
        })

        # Sort by 'Actions' and select the top 5
        frame = frame.sort_values(by='Actions', ascending=False).head(5)

        # Plot the top 5 projects
        plt.figure(figsize=(12, 8))
        plt.barh(frame['Document'], frame['Actions'], color='skyblue')
        plt.title('Top 5 Projects by Actions')
        plt.xlabel('Number of Actions')

        for index, value in enumerate(frame['Actions']):
            plt.text(value, index, str(value), va='center')

        plt.show()

    def update_main_info(self):
        totalProjects = len(df['Document'].unique())
        TotalActions = len(df)
        TotalEmployees = len(df['User'].unique())

        self.basic_stats.value = f"""
        <div style="display: flex; justify-content: center; align-items: center; gap: 10px; padding: 10px;">
            <div style="display: grid; grid-template-rows: auto auto; grid-template-columns: auto auto; align-items: center; width: 40px; height: 40px; ">
                <div style="grid-row: span 2; grid-column: 1; font-size: 30px;">📁</div>
                <div style="grid-row: 1; grid-column: 2; font-size: 18px; font-weight: bold; margin: 0; padding: 0; ">{totalProjects}</div>
                <div style="grid-row: 2; grid-column: 2; margin-left: 5px; font-size: 16px; align-self: start; font-weight: bold; margin: 0; padding: 0;">Projects</div>
            </div>
            <div style="padding:10px 60px 40px;"> </div>
            <div style="display: grid; grid-template-rows: auto auto; grid-template-columns: auto auto; align-items: center; width: 40px; height: 40px; ">
                <div style="grid-row: span 2; grid-column: 1; font-size: 30px;">⚙️</div>
                <div style="grid-row: 1; grid-column: 2; font-size: 18px; font-weight: bold; margin: 0; padding: 0; ">{TotalActions}</div>
                <div style="grid-row: 2; grid-column: 2; margin-left: 5px; font-size: 16px; align-self: start; font-weight: bold; margin: 0; padding: 0;">Actions</div>
            </div>
            <div style="padding:10px 60px 40px;"> </div>
            <div style="display: grid; grid-template-rows: auto auto; grid-template-columns: auto auto; align-items: center; width: 40px; height: 40px; ">
                <div style="grid-row: span 2; grid-column: 1; font-size: 30px;">👥</div>
                <div style="grid-row: 1; grid-column: 2; font-size: 18px; font-weight: bold; margin: 0; padding: 0; ">{TotalEmployees}</div>
                <div style="grid-row: 2; grid-column: 2; margin-left: 5px; font-size: 16px; align-self: start; font-weight: bold; margin: 0; padding: 0;">Employees</div>
            </div>
        </div>
        <p><p>

        """

# Projects class

In [7]:
#@title helper functions

def count_actions(project_name):
    actions_count = df[df['Document'] == project_name].shape[0]
    return actions_count

def calculate_modeling_time(project_name):
    project_df = df[df['Document'] == project_name]
    start_time = project_df['Time'].min()
    end_time = project_df['Time'].max()
    modeling_time = end_time - start_time
    return modeling_time

def last_edited_date(project_name):
    project_df = df[df['Document'] == project_name]
    last_edited = project_df['Time'].max()
    return last_edited


In [8]:
#@title Projects main code
class ProjectsPage(PageBase):
    def create_widgets(self):
        self.project_dropdown = widgets.Dropdown(
            options=df['Document'].unique(),
            description='Project:',
            disabled=False,
            layout=widgets.Layout(margin='20px 10px')
        )

        self.project_title = widgets.HTML(value="")

        self.basic_stats = widgets.HTML(value="")

        self.output_grid = [[widgets.Output() for _ in range(2)] for _ in range(3)]

        self.grid = widgets.GridBox(
            children=[output for row in self.output_grid for output in row],
            layout=widgets.Layout(
                grid_template_columns="repeat(2, 1fr)",
                grid_template_rows="repeat(4, auto)",
                grid_gap="20px"
            )
        )
        self.project_dropdown.observe(self.on_project_change, names='value')
        self.container.children = [self.project_dropdown, self.project_title, self.basic_stats, self.grid]

    def on_project_change(self, change):
        project_name = change.new

        # Update project title and stats
        self.project_title.value = f"<div style=\"text-align: center; font-size: 16px; font-family: sans-serif;\"> <h2>Project: {project_name}</h2> </div>"
        self.update_project_stats(project_name)



        with self.output_grid[0][0]:
            clear_output(wait=True)
            modeling_time = calculate_modeling_time(project_name)
            last_edit = last_edited_date(project_name)
            display(widgets.HTML(f"""<div style="font-size: 16px; background-color: #f7fafc; width: 800px; height: 60px; margin: 0 auto; display: flex; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
              <div style="color: black;">Overall modeling time: {modeling_time}<br>
                It was last edited on: {last_edit}

              </div>
              </div>"""))

        # with self.output_grid[2][0]:
        #     clear_output(wait=True)



        with self.output_grid[1][0]:
            clear_output()
            html1=self.create_chartjs_bar_chart(project_name, "actions_per_tab")
            display(widgets.HTML(value=html1))

        with self.output_grid[1][1]:
            clear_output()
            html2=self.create_chartjs_pie_chart(project_name, "user_action_distribution")
            display(widgets.HTML(value=html2))

        with self.output_grid[2][0]:
            clear_output()
            html3=self.create_chartjs_line_chart(project_name, "modeling_time_per_date")
            display(widgets.HTML(value=html3))

        with self.output_grid[2][1]:
            clear_output()
            html4 = self.create_operations_bar_chart(project_name)
            display(widgets.HTML(value=html4))


    def update_project_stats(self, project_name):
        num_employees = df[df['Document'] == project_name]['User'].nunique()
        num_actions = df[df['Document'] == project_name].shape[0]
        self.basic_stats.value= f"""<div style="display: flex; justify-content: center; align-items: center; gap: 10px; padding: 10px;">
        <div style="display: grid; grid-template-rows: auto auto; grid-template-columns: auto auto; align-items: center; width: 40px; height: 40px; ">
          <div style="grid-row: span 2; grid-column: 1; font-size: 30px;">👥</div>
          <div style="grid-row: 1; grid-column: 2; font-size: 18px; font-weight: bold; margin: 0; padding: 0; ">{num_employees}</div>
          <div style="grid-row: 2; grid-column: 2; margin-left: 5px; font-size: 16px; align-self: start; font-weight: bold; margin: 0; padding: 0;">Employees</div>
        </div>
        <div style="padding:10px 60px 40px;"> </div>
        <div style="display: grid; grid-template-rows: auto auto; grid-template-columns: auto auto; align-items: center; width: 40px; height: 40px; ">
          <div style="grid-row: span 2; grid-column: 1; font-size: 30px;">🖥️</div>
          <div style="grid-row: 1; grid-column: 2; font-size: 18px; font-weight: bold; margin: 0; padding: 0; ">{num_actions}</div>
          <div style="grid-row: 2; grid-column: 2; margin-left: 5px; font-size: 16px; align-self: start; font-weight: bold; margin: 0; padding: 0;">Actions</div>
        </div>

  </div>
  """




    def get_operations_data(self, project_name):
      # Function to check if a description contains any of the specified keywords
      def is_undo_cancel_delete(description):
        return any(word in description.lower() for word in ["undo", "cancel", "delete"])

      def is_insert_feature(description):
        return "insert feature" in description.lower()

      def is_edit(description):
        return "edit" in description.lower()


      # Filter data for the specific project
      project_data = df[df['Document'] == project_name].copy()
      project_data['Date'] = project_data['Time'].dt.date

      # Group by date and count occurrences of each type of action
      operation_counts = project_data.groupby('Date')['Description'].apply(
          lambda x: pd.Series({
              'Delete_Cancel_Undo': x.apply(is_undo_cancel_delete).sum(),
              'Insert_Feature': x.apply(is_insert_feature).sum(),
              'Edit': x.apply(is_edit).sum()
          })
      ).reset_index()

      # Ensure all columns are present even if they have zero counts
      for op_type in ['Delete_Cancel_Undo', 'Insert_Feature', 'Edit']:
          if op_type not in operation_counts.columns:
              operation_counts[op_type] = 0

      return operation_counts



    def get_labels(self, project_name, chart_type):
        if chart_type == "actions_per_tab":
            project_data = df[df['Document'] == project_name]
            return project_data['Tab'].value_counts().index.tolist()
        elif chart_type == "user_action_distribution":
            project_data = df[df['Document'] == project_name]
            return project_data['User'].value_counts().index.tolist()
        elif chart_type == "modeling_time_per_date":
            project_data = df[df['Document'] == project_name].copy()
            project_data['Date'] = project_data['Time'].dt.date
            return [date.strftime('%Y-%m-%d') for date in project_data.groupby('Date').size().index]
        return []

    def get_data(self, project_name, chart_type):
      if chart_type == "actions_per_tab":
          project_data = df[df['Document'] == project_name]
          return project_data['Tab'].value_counts().tolist()
      elif chart_type == "user_action_distribution":
          project_data = df[df['Document'] == project_name]
          counts = project_data['User'].value_counts()
          total = counts.sum()
          return (counts / total * 100).tolist()  # Convert to percentages
      elif chart_type == "modeling_time_per_date":
          project_data = df[df['Document'] == project_name].copy()
          project_data['Date'] = project_data['Time'].dt.date
          return (project_data.groupby('Date')
                  .apply(lambda x: (x['Time'].max() - x['Time'].min()).total_seconds() / 3600)
                  .tolist())
      return []



    def create_chartjs_pie_chart(self, project_name, chart_type):
        labels = self.get_labels(project_name, chart_type)
        data = self.get_data(project_name, chart_type)
        if len(labels) != len(data):
          raise ValueError("Labels and data length mismatch.")
        html = f"""
        <div style=" background-color: #f7fafc; width: 800px; height: 400px; margin: 0 auto; display: flex; flex-direction: column; justify-content: center; align-items: center; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
            <div><h2>Percentage of Actions per User </h2></div>
            <canvas id="pieChart" width="400" height="200"></canvas>
            <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
            <script>
                var ctx = document.getElementById('pieChart').getContext('2d');
                var myPieChart = new Chart(ctx, {{
                    type: 'pie',
                    data: {{
                        labels: {json.dumps(labels)},
                        datasets: [{{
                            data: {json.dumps(data)},
                            backgroundColor: ['#FF6384', '#36A2EB', '#FFCE56'],
                        }}]
                    }},
                    options: {{
                        responsive: true,
                        plugins: {{
                            legend: {{
                                position: 'left',
                            }},
                            tooltip: {{
                                callbacks: {{
                                    label: function(context) {{
                                        let label = context.label || '';
                                        if (context.parsed) {{
                                            label += ': ' + context.parsed + '%';
                                        }}
                                        return label;
                                    }}
                                }}
                            }}
                        }}
                    }}
                }});
            </script>
        </div>
        """
        return html


    def create_chartjs_bar_chart(self, project_name, chart_type):
        labels = self.get_labels(project_name, chart_type)
        data = self.get_data(project_name, chart_type)
        html = f"""
        <div style=" background-color: #f7fafc; width: 800px; height: 400px; margin: 0 auto; display: flex; flex-direction: column; justify-content: center; align-items: center; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
            <div><h2>Actions number per Tab</h2></div>
            <canvas id="barChart" width="400" height="200"></canvas>
            <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
            <script>
                var ctx = document.getElementById('barChart').getContext('2d');
                var myBarChart = new Chart(ctx, {{
                    type: 'bar',
                    data: {{
                        labels: {json.dumps(labels)},
                        datasets: [{{
                            label: 'Number of Actions',
                            data: {json.dumps(data)},
                            backgroundColor: 'rgba(75, 192, 192, 0.4)',
                            borderColor: 'rgba(75, 192, 192, 1)',
                            borderWidth: 1
                        }}]
                    }},
                    options: {{
                        scales: {{
                            x: {{
                                beginAtZero: true,
                                title: {{
                                  display: true,
                                  text: 'Tab Name' // Your x-axis label
                              }}
                            }},
                            y: {{
                                beginAtZero: true
                            }}
                        }},
                        responsive: true
                    }}
                }});
            </script>
        </div>
        """
        return html


    def create_chartjs_line_chart(self, project_name, chart_type):
        labels = self.get_labels(project_name, chart_type)
        data = self.get_data(project_name, chart_type)
        html = f"""
        <div style=" background-color: #f7fafc; width: 800px; height: 400px; margin: 0 auto; display: flex; flex-direction: column; justify-content: center; align-items: center; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
            <div><h2>Modeling time per Date </h2></div>
            <canvas id="lineChart" width="400" height="200" ></canvas>
            <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
            <script>
                var ctx = document.getElementById('lineChart').getContext('2d');
                var myLineChart = new Chart(ctx, {{
                    type: 'line',
                    data: {{
                        labels: {json.dumps(labels)},
                        datasets: [{{
                            label: 'Modeling Time',
                            data: {json.dumps(data)},
                            fill: false,
                            borderColor: 'rgba(255, 159, 64, 1)',
                            tension: 0.1
                        }}]
                    }},
                    options: {{
                        responsive: true,
                        scales: {{
                            x: {{
                                title: {{
                                    display: true,
                                    text: 'Date'
                                }}
                            }},
                            y: {{
                                title: {{
                                    display: true,
                                    text: 'Modeling Time (Hours)'
                                }},
                                beginAtZero: true
                            }}
                        }}
                    }}
                }});
            </script>
        </div>
        """
        return html



    def get_operations_data(self, project_name):
      # Function to check if a description contains any of the specified keywords
      def is_undo_cancel_delete(description):
        result= any(word in description.lower() for word in ["undo", "cancel", "delete"])


        return result

      def is_insert_feature(description):
        return "insert feature" in description.lower()

      def is_edit(description):
        return "edit" in description.lower()


      # Filter data for the specific project
      project_data = df[df['Document'] == project_name].copy()
      project_data['Date'] = project_data['Time'].dt.date

       # Create new columns for each action type
      project_data['Delete_Cancel_Undo'] = project_data['Description'].apply(is_undo_cancel_delete)
      project_data['Insert_Feature'] = project_data['Description'].apply(is_insert_feature)
      project_data['Edit'] = project_data['Description'].apply(is_edit)

      # Group by date and count occurrences of each type of action
      operation_counts = project_data.groupby('Date').agg({
          'Delete_Cancel_Undo': 'sum',
          'Insert_Feature': 'sum',
          'Edit': 'sum'
      }).reset_index()

      return operation_counts




    def create_operations_bar_chart(self, project_name):
        operation_counts = self.get_operations_data(project_name)

        labels = operation_counts['Date'].astype(str).tolist()
        delete_cancel_undo_data = operation_counts['Delete_Cancel_Undo'].tolist()
        insert_feature_data = operation_counts['Insert_Feature'].tolist()
        edit_data = operation_counts['Edit'].tolist()


        html = f"""
        <div style=" background-color: #f7fafc; width: 800px; height: 400px; margin: 0 auto; display: flex; flex-direction: column; justify-content: center; align-items: center; padding: 20px; border-radius: 15px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);">
            <div><h2>Operations Count per Date</h2></div>
            <canvas id="operationsBarChart" width="400" height="200"></canvas>
            <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
            <script>
                var ctx = document.getElementById('operationsBarChart').getContext('2d');
                var myBarChart = new Chart(ctx, {{
                    type: 'bar',
                    data: {{
                        labels: {json.dumps(labels)},
                        datasets: [
                            {{
                                label: 'Delete/Cancel/Undo',
                                data: {json.dumps(delete_cancel_undo_data)},
                                backgroundColor: 'rgba(255, 99, 132, 0.2)',
                                borderColor: 'rgba(255, 99, 132, 1)',
                                borderWidth: 1
                            }},
                            {{
                                label: 'Insert Feature',
                                data: {json.dumps(insert_feature_data)},
                                backgroundColor: 'rgba(54, 162, 235, 0.2)',
                                borderColor: 'rgba(54, 162, 235, 1)',
                                borderWidth: 1
                            }},
                            {{
                                label: 'Edit',
                                data: {json.dumps(edit_data)},
                                backgroundColor: 'rgba(75, 192, 192, 0.2)',
                                borderColor: 'rgba(75, 192, 192, 1)',
                                borderWidth: 1
                            }}
                        ]
                    }},
                    options: {{
                      responsive: true,
                        scales: {{
                            x: {{
                                beginAtZero: true,
                                title: {{
                                    display: true,
                                    text: 'Date'
                                }}
                            }},
                            y: {{
                                beginAtZero: true,
                                title: {{
                                    display: true,
                                    text: 'Operation Count'
                                }}
                            }}
                        }}

                    }}
                }});
            </script>
        </div>
        """
        return html




# Employees Page class

In [9]:


# Define the categories and their associated keywords
category_1_keywords = ["open", "close", "import", "export", "create document", "insert tab", "rename", "update version", "show", "hide"]
category_2_keywords = ["add", "delete", "insert feature", "commit add", "create folder", "create version"]
category_3_keywords = ["edit", "modify", "update", "change", "commit edit", "start edit"]

def classify_description(description):
    description_lower = description.lower()
    if any(keyword in description_lower for keyword in category_1_keywords):
        return 'Category 1'
    elif any(keyword in description_lower for keyword in category_2_keywords):
        return 'Category 2'
    elif any(keyword in description_lower for keyword in category_3_keywords):
        return 'Category 3'
    else:
        return 'Uncategorized'

# Apply classification to the DataFrame
df['Category'] = df['Description'].apply(classify_description)

# Group by User, Date, and Category and count the number of actions
df['Date'] = df['Time'].dt.date

def count_categories(student):
    # Filter the DataFrame for the given student
    student_df = df[df['User'] == student].copy()
    # Create a new column for month-year
    student_df['MonthYear'] = student_df['Time'].dt.to_period('M')
    # Get the range of months/years for the student
    st = student_df['MonthYear'].min()
    end = student_df['MonthYear'].max()

    # Create a date range from start to end months
    date_range = pd.period_range(start=st, end=end, freq='M')

    # Initialize a map to store counts for each category per month-year
    category_map = {date: [0, 0, 0] for date in date_range}

    # Iterate over each row in the filtered DataFrame
    for index, row in student_df.iterrows():
        month_year = row['MonthYear']
        category = row['Category']

        # Update the counts for the specific category
        if category == "Category 1":
            category_map[month_year][0] += 1
        elif category == "Category 2":
            category_map[month_year][1] += 1
        elif category == "Category 3":
            category_map[month_year][2] += 1

    # Convert category_map to DataFrame for plotting
    plot_df = pd.DataFrame(category_map).T
    plot_df.index = plot_df.index.to_timestamp()  # Convert PeriodIndex to TimestampIndex

    # Plot the data
    plt.figure(figsize=(12, 6))
    plt.plot(plot_df.index, plot_df[0], label='Basic Document Operations', color='blue', marker='o')
    plt.plot(plot_df.index, plot_df[1], label='Adding and Deleting Content', color='green', marker='o')
    plt.plot(plot_df.index, plot_df[2], label='Editing and Modifying Content', color='red', marker='o')

    plt.xlabel('Date')
    plt.ylabel('Number of Operations')
    plt.title(f'Number of Operations per Category for {student}')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return category_map

# Define functions for additional data analysis
def countCreatForUser(user):
    filtered_df = df[df['User'] == user]
    dscrptns = filtered_df['Description']
    cnt = 0
    st = "Create"
    st2 = "created"
    for i in dscrptns:
        splited = i.split(" ")
        if splited[0] == st:
            cnt += 1
        for j in splited:
            if j == st2:
                cnt += 1
    return cnt

def countDeletsForUser(user):
    filtered_df = df[df['User'] == user]
    dscrptns = filtered_df['Description']
    cnt = 0
    st = "Delete"
    for i in dscrptns:
        splited = i.split(" ")
        if splited[0] == st:
            cnt += 1
    return cnt

def countAddOrInsertForUser(user):
    filtered_df = df[df['User'] == user]
    dscrptns = filtered_df['Description']
    cnt = 0
    for i in dscrptns:
        splited = i.split(" ")
        if splited[0] == "Add" or splited[0] == "Insert":
            cnt += 1
    return cnt

def countProjectsForUser(user):
    filtered_df = df[df['User'] == user]
    user_counts = filtered_df['Document'].value_counts()
    return user_counts.size

def calcTime(user):
    filtered_df = df[df['User'] == user].drop_duplicates(subset=['Time'])
    times = filtered_df['Time']
    length_of_times = len(times)
    hours = int(length_of_times // 3600)
    minutes = int(length_of_times // 60)
    seconds = int(length_of_times % 60)
    time = f"{hours}:{minutes}:{seconds}"
    return time
class EmployeesPage(PageBase):
    def create_widgets(self):
        # Dropdowns for selecting year, month, and user
        self.year_dropdown = widgets.Dropdown(
            options=df['Time'].dt.year.unique(),
            description='Year:',
            disabled=False,
            layout=widgets.Layout(margin='20px 10px')
        )

        self.month_dropdown = widgets.Dropdown(
            options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
            description='Month:',
            disabled=False,
            layout=widgets.Layout(margin='20px 10px')
        )

        self.user_dropdown = widgets.Dropdown(
            options=df['User'].unique(),
            description='User:',
            disabled=False,
            layout=widgets.Layout(margin='20px 10px')
        )

        # Button to trigger filtering
        self.filter_button = widgets.Button(description='Filter', layout=widgets.Layout(margin='20px 10px'))

        # Output widget for displaying the result and new category operations plot
        self.output = widgets.Output()
        self.category_output = widgets.Output()

        # Initialize student_name with an empty string
        self.student_name = widgets.HTML(value="<h2></h2>")

        # Create data boxes layout and heatmap output widget
        self.data_boxes = []
        self.data_boxes_layout = widgets.GridBox(
            self.data_boxes,
            layout=widgets.Layout(
                grid_template_columns="repeat(5, 1fr)",
                grid_template_areas="""
                    "data1 data2 data3 data4 data5"
                """,
                width='100%',
                grid_gap='15px',
                padding='20px'
            )
        )
        self.heatmap_output = widgets.Output()

        # Create the container and add widgets to it
        self.container = widgets.VBox([
            widgets.HBox([self.year_dropdown, self.month_dropdown, self.user_dropdown, self.filter_button]),
            self.output,
            self.student_name,
            self.data_boxes_layout,
            self.heatmap_output,
            self.category_output  # New category operations output widget
        ])

        # Attach the button click event to the filtering function
        self.filter_button.on_click(self.filter_data)

        # Attach dropdown change events to update data items
        self.user_dropdown.observe(self.user_selection_change, names='value')

    def filter_data(self, b):
        # Get selected year, month, and user
        selected_year = self.year_dropdown.value
        selected_month = self.month_dropdown.value
        selected_user = self.user_dropdown.value

        # Filter the data based on the selected year, month, and user
        filtered_df = df[
            (df['Time'].dt.year == selected_year) &
            (df['Time'].dt.month == selected_month) &
            (df['User'] == selected_user)
        ]

        if filtered_df.empty:
            with self.output:
                clear_output()
                print(f'No data available for {selected_user} in {selected_year}-{selected_month}.')
            return

        # Count actions by type and project
        actions_by_project = filtered_df.groupby('Document').apply(lambda g: pd.Series({
            'Created': g['Description'].str.contains('Create', case=False, na=False).sum(),
            'Edited': g['Description'].str.contains('Edit', case=False, na=False).sum(),
            'Deleted': g['Description'].str.contains('Delete', case=False, na=False).sum()
        })).reset_index()

        # Plot the distribution of actions by day
        daily_actions = filtered_df.groupby(filtered_df['Time'].dt.date).size()
        fig, ax = plt.subplots(figsize=(10, 6))
        daily_actions.plot(kind='bar', ax=ax)
        ax.set_title(f'Actions Distribution by Day for {selected_user} in {selected_year}-{selected_month}')
        ax.set_xlabel('Date')
        ax.set_ylabel('Number of Actions')
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Display results
        with self.output:
            clear_output()
            print(f'Total actions for {selected_user} in {selected_year}-{selected_month}: {filtered_df.shape[0]}')

            # Create and display the table
            table = widgets.HTML(
                value=actions_by_project.to_html(index=False),
                layout=widgets.Layout(overflow_x='auto', width='50%')
            )

            # Display the bar chart in a widget
            plot_output = widgets.Output()
            with plot_output:
                plt.show(fig)

            # Arrange table and graph side by side
            display(widgets.HBox([table, plot_output]))

        # Update student-specific data
        self.update_student_name(selected_user)
        self.update_category_operations(selected_user)  # Update the category operations plot

    def user_selection_change(self, change):
        # Triggered when the user selection changes
        selected_user = change.new
        self.update_student_name(selected_user)
        self.update_category_operations(selected_user)  # Ensure category operations are updated when user changes

    def update_student_name(self, student_name):
        self.student_name.value = f"<h2 style='text-align:center; margin:10px 0; color: #333;'>{student_name}</h2>"
        self.update_data_items(student_name)
        self.update_heatmap(student_name)  # Update the heatmap based on the selected user

    def update_data_items(self, student_name):
        self.data_items = [
            (countProjectsForUser(student_name), "Projects"),
            (calcTime(student_name), "Modeling Time"),
            (countCreatForUser(student_name), "Elements Created"),
            (countAddOrInsertForUser(student_name), "Elements Added/Inserted"),
            (countDeletsForUser(student_name), "Elements Deleted")
        ]

        self.data_boxes = []
        for value, label in self.data_items:
            box = widgets.HTML(f"""
                <div style="
                    background-color: #f8f9fa;
                    border: 1px solid #e9ecef;
                    border-radius: 5px;
                    padding: 15px;
                    text-align: center;
                    height: 100%;
                    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                ">
                    <h3 style="margin:5px; font-size: 24px; color: #333;">{value}</h3>
                    <p style="margin:5px; color: #6c757d;">{label}</p>
                </div>
            """)
            self.data_boxes.append(box)

        self.update_data_boxes()

    def update_data_boxes(self):
        self.data_boxes_layout.children = self.data_boxes

    def update_heatmap(self, user):
        df['Day of Week'] = df['Time'].dt.day_name()
        df['Hour of Day'] = df['Time'].dt.hour

        grouped = df.groupby(['User', 'Day of Week', 'Hour of Day']).size().reset_index(name='Count')

        user_data = grouped[grouped['User'] == user]

        heatmap_data = user_data.pivot(index='Hour of Day', columns='Day of Week', values='Count')

        all_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        all_hours = list(range(24))

        heatmap_data = heatmap_data.reindex(index=all_hours, columns=all_days, fill_value=0)

        heatmap_data = heatmap_data.fillna(0).astype(int)

        # Plot the heatmap
        fig, ax = plt.subplots(figsize=(12, 8))
        sns.heatmap(heatmap_data, cmap='coolwarm', annot=True, fmt="d", ax=ax)
        ax.set_title(f'Heatmap of Number of Actions for {user}')
        ax.set_xlabel('Day of Week')
        ax.set_ylabel('Hour of Day')
        plt.tight_layout()

        with self.heatmap_output:
            clear_output()
            plt.show()

    def update_category_operations(self, user):
        with self.category_output:
            clear_output(wait=True)
            count_categories(user)  # Display the plot for the selected user





# Notifications Page class

In [10]:

class NotificationsPage(PageBase):
    def create_widgets(self):
        self.content = widgets.HTML(value='<h1>Notifications Page Content</h1>')
        self.container.children = [self.content]
        self.data = data
        # Convert string time to datetime object
        for entry in self.data:
          if isinstance(entry['Time'], str):
            entry['Time'] = datetime.strptime(entry['Time'], '%Y-%m-%d %H:%M:%S')

        # Sort data by time (oldest to newest)
        self.data.sort(key=lambda x: x['Time'])

        # Sort data by time (oldest to newest)
        self.data.sort(key=lambda x: x['Time'])


        # Helper function to detect undo/cancel/delete actions
        def is_undo_cancel_delete(description):
            return any(word in description.lower() for word in ["undo", "cancel", "delete"])

        # Function to detect Case 1
        def detect_case_1(data):
            notifications = []
            notified_combinations = set()
            for i, entry in enumerate(data):
                for j in range(i + 1, len(data)):
                    if (data[j]['Time'] - entry['Time']) > timedelta(minutes=5):
                        break
                    if entry['Document'] == data[j]['Document'] and entry['Tab'] == data[j]['Tab'] and entry['User'] != data[j]['User']:
                        combination = (entry['Document'], entry['Tab'], entry['User'], data[j]['User'])
                        if combination not in notified_combinations and combination[::-1] not in notified_combinations:
                            notification = {
                                "Time": data[j]['Time'],
                                "Title": "Heightened conflict chance.",
                                "Body": f"Your employees {entry['User']} & {data[j]['User']} are working simultaneously on Project \"{entry['Document']}\", Tab \"{entry['Tab']}\"."
                            }
                            notifications.append(notification)
                            notified_combinations.add(combination)
            return notifications

        # Function to detect Case 2
        def detect_case_2(data):
            notifications = []
            action_count = {}
            for entry in data:
                if is_undo_cancel_delete(entry['Description']):
                    user = entry['User']
                    if user not in action_count:
                        action_count[user] = []
                    action_count[user].append(entry['Time'])

                    # Remove actions older than 5 minutes
                    action_count[user] = [time for time in action_count[user] if entry['Time'] - time <= timedelta(minutes=5)]

                    if len(action_count[user]) > 5:
                        notification = {
                            "Time": entry['Time'],
                            "Title": "Your employee might need help.",
                            "Body": f"Your employee {user} is backing from a number of operations. You might want to consider setting up another team member for help."
                        }
                        notifications.append(notification)
                        action_count[user] = []
            return notifications

        # Function to detect Case 3
        def detect_case_3(data):
            notifications = []
            last_worked = {}
            current_time = datetime.now()
            cutoff_time = current_time - timedelta(weeks=2)

            for entry in data:
                project = entry['Document']
                if project not in last_worked or entry['Time'] > last_worked[project]:
                    last_worked[project] = entry['Time']

            for project, last_time in last_worked.items():
                if last_time < cutoff_time:
                    notification = {
                        "Time": last_time,
                        "Title": "Ditched Project",
                        "Body": f"\"{project}\" hasn’t had any work done for more than two weeks. Consider taking a look."
                    }
                    notifications.append(notification)
            return notifications

        # Function to detect Case 4
        def detect_case_4(data):
            notifications = []
            project_actions = {}
            for entry in data:
                if is_undo_cancel_delete(entry['Description']):
                    project = entry['Document']
                    if project not in project_actions:
                        project_actions[project] = []
                    project_actions[project].append(entry['Time'])

                    # Remove actions older than 5 minutes
                    project_actions[project] = [time for time in project_actions[project] if entry['Time'] - time <= timedelta(minutes=5)]

                    if len(project_actions[project]) > 5:
                        notification = {
                            "Time": entry['Time'],
                            "Title": "Your project might be facing difficulties.",
                            "Body": f"\"{project}\" has had multiple undo/cancel/delete operations recently. Consider taking a look."
                        }
                        notifications.append(notification)
                        project_actions[project] = []
            return notifications

        def filter_notifications(notifications):
            filtered_notifications = []
            seen_bodies = {}

            for notification in notifications:
                body = notification['Body']
                time = notification['Time']

                if body not in seen_bodies:
                    seen_bodies[body] = time
                    filtered_notifications.append(notification)
                else:
                    last_time = seen_bodies[body]
                    if time - last_time > timedelta(hours=1):
                        seen_bodies[body] = time
                        filtered_notifications.append(notification)

            return filtered_notifications

        # Combine notifications from all cases and order by date
        def combine_notifications(data):
            notifications = []
            notifications.extend(detect_case_1(data))
            notifications.extend(detect_case_2(data))
            notifications.extend(detect_case_3(data))
            notifications.extend(detect_case_4(data))

            # Sort notifications by time
            notifications.sort(key=lambda x: x['Time'], reverse=True)

            # Filter notifications to keep only one within an hour for the same body
            filtered_notifications = filter_notifications(notifications)

            return filtered_notifications

        # Get combined and filtered notifications
        notifications = combine_notifications(self.data)


        # Create notification widgets and add to the container
        notification_widgets = []
        for notification in notifications:
            notification_html = widgets.HTML(
                value=f"""
                <div style="border: 1px solid #ccc; padding: 15px; margin: 10px 0; background-color: #f4f4f4; border-radius: 8px; display: flex; align-items: center; justify-content: space-between;">
                  <div style="flex: 4; padding-left: 20px;">
                      <strong style="font-size: 1.2em; display: block; margin-bottom: 5px; color: #333;">
                          {notification['Title']}
                      </strong>
                      <p style="margin: 0; color: #555;">{notification['Body']}</p>
                  </div>
                  <div style="font-size: 0.8em; color: #666; min-width: 100px; text-align: left; align-self: flex-start;">
                      {notification['Time'].strftime('%Y-%m-%d %H:%M:%S')}
                  </div>
              </div>
                """,
                layout=widgets.Layout(width='100%')
            )
            notification_widgets.append(notification_html)

        # Update the container with the new notification widgets
        self.container.children = [self.content] + notification_widgets

# UploadJson Page class




In [11]:
#@title new upload json page class

class UploadJsonPage(PageBase):
    def create_widgets(self):
        self.content = widgets.HTML(value='<h1>UploadJson Page Content</h1>')

        # Create the file upload widget
        self.upload_widget = widgets.FileUpload(
            accept='.json',  # Accept only .json files
            multiple=False   # Single file upload
        )

        # Create a dropdown to select from existing JSON files
        self.dropdown = widgets.Dropdown(
            options=self.get_existing_json_files(),
            description='Select file:',
            disabled=False,
        )

        # Create a button to set the selected file
        self.set_button = widgets.Button(description='Set as Data')
        self.set_button.on_click(self.on_set_button_click)

        # Create an output widget to display messages
        self.output = widgets.Output()

        # Attach the upload handler to the widget
        self.upload_widget.observe(self.on_upload_change, names='value')

        self.container.children = [self.content, self.upload_widget, self.dropdown, self.set_button, self.output]

    def get_existing_json_files(self):
        # Fetch existing JSON files from Firebase
        existing_files = FBconn.get('/JsonFiles/', None)
        if existing_files:
            return [value['file_name'] for key, value in existing_files.items()]
        return []

    def on_set_button_click(self, b):
        selected_file_name = self.dropdown.value
        if selected_file_name:
            # Instead of using the key, use the file name to retrieve the file data
            file_data, file_df = prep_data(selected_file_name)

            if file_data:
                global data
                global df
                global cur_file_name
                # data = pd.DataFrame(file_data['data'])
                # data['Time'] = pd.to_datetime(data['Time'])
                # data['Document'] = data['Document'].fillna('Unknown')

                data, df = file_data, file_df
                cur_file_name = selected_file_name


                # Update the header in the main dashboard with the selected file name
                dashboard.refresh_dashboard()
                with self.output:
                    self.output.clear_output()
                    print(f"Data set to {selected_file_name}")

    def on_upload_change(self, change):
        self.output.clear_output()
        with self.output:
            # Check if a file has been uploaded
            if len(self.upload_widget.value) > 0:
                # Get the uploaded file
                file_info = list(self.upload_widget.value.values())[0]
                file_name = file_info['metadata']['name']
                file_content = file_info['content']

                # Check if the uploaded file is a JSON
                try:
                    json_data = json.loads(file_content.decode('utf-8'))
                    print(f"{file_name} is a valid JSON file.")

                    ################# check if valid json file



                    exist, updatekey = self.file_exists_in_firebase(file_name)
                    if exist:
                        # Create a button widget for the popup
                        update_button = widgets.Button(description="Update")
                        cancel_button = widgets.Button(description="Cancel")

                        def on_update_click(b):
                            result = self.update_json_in_firebase(file_name, file_content, updatekey)
                            print("File successfully updated in Firebase.")
                            print(result)
                            popup.close()
                            # Refresh the dropdown options after update
                            self.dropdown.options = self.get_existing_json_files()

                        def on_cancel_click(b):
                            print("Upload cancelled.")
                            popup.close()

                        update_button.on_click(on_update_click)
                        cancel_button.on_click(on_cancel_click)

                        # Create the popup widget
                        popup = widgets.VBox([
                            widgets.HTML(f"<b>{file_name}</b> already exists. Do you want to update it?"),
                            widgets.HBox([update_button, cancel_button])
                        ])

                        display(popup)
                    else:
                        result = self.upload_json_to_firebase(file_name, file_content)
                        print("File successfully uploaded to Firebase.")
                        print(result)
                        # Refresh the dropdown options after upload
                        self.dropdown.options = self.get_existing_json_files()

                except json.JSONDecodeError:
                    print(f"{file_name} is not a valid JSON file.")




# ChatBot Page class

In [12]:
#@title chatbot logic

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Function to generate dynamic response - kkkkk
def generate_user_actions_response(user_input):
    match = re.match(r'what are the 5 top actions (.*) performed\?', user_input)
    if match:
        user = match.group(1)
        # Filter the data to get only the actions performed by the specified user
        print("user is")
        print(user)
        user_actions = [entry['Description'] for entry in data if entry['User'] == user]

        if user_actions:
            # Step 1: Count the occurrences of each action
            action_counts = Counter(user_actions)

            # Step 2: Get the top 5 actions with the highest counts
            top_5_actions = action_counts.most_common(5)

            # Step 3: Format the output string with numbering
            output_string = "Top 5 actions:\n"
            for i, (action, count) in enumerate(top_5_actions, 1):
                output_string += f"{i}. {action} ({count} times)\n"
            return output_string
        else:
            return f"No actions recorded for user '{user}'"
    return None

def generate_documents_accessed_response(user_input):
    match = re.match(r'can you list all the documents accessed by (.*)\?', user_input)
    if match:
        user = match.group(1)
        # Filter the data to get only the documents accessed by the specified user
        accessed_documents = set(entry['Document'] for entry in data if entry['User'] == user and 'Document' in entry)

        if accessed_documents:
            documents_list = '\n '.join(accessed_documents)
            return f"{user} accessed the following documents: {documents_list}"
        else:
            return f"No documents accessed by user '{user}'"
    return None


def generate_tab_access_response(user_input):
    match = re.match(r'which user accessed the (.*) tab the most\?', user_input)
    if match:
        tab = match.group(1)
        # Logic to find the user who accessed the specified tab the most
        if tab in tabs:
            # Assuming you have a data structure that maps users to tab access counts
            # For example: {'Tab1': {'User1': 10, 'User2': 5}}
            tab_users = {user: count for user, count in users.items() if tab in tabs}
            if tab_users:
                most_common_user = max(tab_users, key=tab_users.get)
                accesses = tab_users[most_common_user]
                return f"The user who accessed the {tab} tab the most is {most_common_user} with {accesses} accesses."
            else:
                return f"No user has accessed the {tab} tab."
        else:
            return f"No data available for the tab '{tab}'."
    return None

def generate_least_used_tab_response(user_input):
    match = re.match(r'which tab was used the least by (.*)\?', user_input)
    if match:
        user = match.group(1)
        # Filter the data to get only the entries relevant to the specified user
        user_tabs = defaultdict(int)
        for entry in data:
            if entry['User'] == user and entry['Tab'] != 'N/A':
                user_tabs[entry['Tab']] += 1

        if user_tabs:
            # Find the least used tab by the user
            least_used_tab = min(user_tabs, key=user_tabs.get)
            uses = user_tabs[least_used_tab]
            return f"The least used tab by {user} is '{least_used_tab}' with only {uses} uses."
        else:
            return f"{user} has not used any tabs or all actions are associated with 'N/A' tabs."
    return None



# Custom chatbot function - kkkkk
def custom_chatbot(user_input):
    if re.match(r'what are the 5 top actions (.*) performed\?', user_input):
        return generate_user_actions_response(user_input)
    elif re.match(r'can you list all the documents accessed by (.*)\?', user_input):
        return generate_documents_accessed_response(user_input)
    elif re.match(r'which user accessed the (.*) tab the most\?', user_input):
        return generate_tab_access_response(user_input)
    elif re.match(r'which tab was used the least by (.*)\?', user_input):
        return generate_least_used_tab_response(user_input)
    else:
        return chatbot.respond(user_input)


# Process the data
users = Counter([item.get('User') for item in data])
documents = Counter([item.get('Document') for item in data if item.get('Document')])
tabs = Counter([item.get('Tab') for item in data if item.get('Tab')])
descriptions = Counter([item.get('Description') for item in data if item.get('Description')])

# Calculate some statistics
total_actions = len(data)
unique_users = len(users)
unique_documents = len(documents)
unique_tabs = len(tabs)

# Define patterns and responses
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Welcome to the project management assistant.']),
    (r'how are you?', ['I\'m functioning well, thank you!', 'I\'m operational and ready to assist with your project management.']),
    (r'what is your name?', ['I\'m the Project Management Assistant.', 'You can call me the Assistant.']),
    (r'how do I use context in my project?', ['You can use context to manage state and share data across different parts of your application.']),
    (r'how do I define a type?', ['Defining types helps ensure that your variables are used correctly and can prevent errors in your application.']),
    (r'how can I use a keyboard shortcut?', ['Keyboard shortcuts can be defined to improve the efficiency of your application, allowing users to perform actions quickly.']),
    (r'how do I plan my project?', ['Planning your project involves setting clear goals, defining tasks, and creating a timeline to ensure that everything is completed on time.']),
    (r'how do I assemble my project components?', ['Assembling your project components involves integrating different parts of your application to work together seamlessly.']),
    (r'how can I sketch my ideas?', ['Sketching your ideas can help visualize your project before implementation, making it easier to plan and execute.']),
    (r'how do I draw diagrams for my project?', ['Drawing diagrams can help illustrate the structure and flow of your application, making it easier to understand and communicate.']),
    (r'what is a part in project management?', ['A part in project management refers to a component or segment of the overall project, which can be managed and tracked individually.']),
    (r'what is a studio in the context of development?', ['A studio in development refers to an integrated environment where various aspects of a project can be designed, developed, and tested.']),
    (r'exit|bye|goodbye', ['Thank you for using the Project Management Assistant. Goodbye!', 'Farewell! Don\'t hesitate to return if you need more assistance with your project.']),
    (r'how many total actions are recorded?', [f"There are {total_actions} total actions recorded in the database."]),
    (r'who is the most active user?', [f"The most active user is {users.most_common(1)[0][0]} with {users.most_common(1)[0][1]} actions."]),
    (r'how many unique users are there?', [f"There are {unique_users} unique users in the database."]),
    (r'What is the most frequently accessed document?', [f"The most frequently accessed document is '{documents.most_common(1)[0][0]}' with {documents.most_common(1)[0][1]} accesses."]),
    (r'how many different documents were accessed?', [f"There were {unique_documents} different documents accessed."]),
    (r'what is the most common tab used?', [f"The most commonly used tab is '{tabs.most_common(1)[0][0]}' with {tabs.most_common(1)[0][1]} uses."]),
    (r'what is the most frequent action description?', [f"The most frequent action description is '{descriptions.most_common(1)[0][0]}' occurring {descriptions.most_common(1)[0][1]} times."]),
    (r'what are the top 3 most active users?', [f"The top 3 most active users are: 1. {users.most_common(3)[0][0]} ({users.most_common(3)[0][1]} actions), 2. {users.most_common(3)[1][0]} ({users.most_common(3)[1][1]} actions), and 3. {users.most_common(3)[2][0]} ({users.most_common(3)[2][1]} actions)."]),
    (r'what is the least used tab?', [f"The least used tab is '{tabs.most_common()[-1][0]}' with only {tabs.most_common()[-1][1]} uses."]),
    (r'how many different types of actions \(descriptions\) are there?', [f"There are {len(descriptions)} different types of actions (unique descriptions) in the database."]),

    (r'what are the 5 top actions (.*) performed\?', 'DYNAMIC_RESPONSE'),
     (r'can you list all the documents accessed by (.*)\?', 'DYNAMIC_RESPONSE'),
    (r'which user accessed the (.*) tab the most\?', 'DYNAMIC_RESPONSE'),
    (r'which tab was used the least by (.*)\?', 'DYNAMIC_RESPONSE')
]

# Create a chatbot
chatbot = Chat(patterns, reflections)

def process_input(user_input):

    if user_input.lower() in ['exit', 'bye', 'goodbye']:
        return "Thank you for using the Project Management Assistant. Goodbye!"

    response = custom_chatbot(user_input)
    return response if response is not None else "This type of question is not supported."


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [13]:
#@title Chatbot frontend
class ChatBotPage(PageBase):
    def create_widgets(self):
        # Create widgets for the ChatBot page
        self.chatbot_title = widgets.HTML("<h2>ChatBot</h2>")

        # Create suggestion words in a dropdown menu
        self.suggestion_words = []
        for pattern, _ in patterns:
            self.suggestion_words.append(pattern)
        self.suggestions_dropdown = widgets.Dropdown(
            options=['Select a suggestion'] + self.suggestion_words,
            value='Select a suggestion',
            description='Suggestions:',
            style={'description_width': 'initial','font_size': '1rem'},
            layout=widgets.Layout( font_size='20px')
        )
        self.suggestions_dropdown.observe(self.on_suggestion_select, names='value')

        # Chat input and output
        self.chat_input = widgets.Text(placeholder='Type your message here...',layout=widgets.Layout(width='auto', font_size='60px'))
        self.chat_output = widgets.VBox()
        self.send_button = widgets.Button(description="Send",layout=widgets.Layout(width='auto', font_size='20px'))
        self.send_button.on_click(self.on_send_click)

        # Layout the widgets
        self.container = widgets.VBox([
            self.chatbot_title,
            self.suggestions_dropdown,  # Add the suggestions dropdown
            self.chat_output,
            self.chat_input,
            self.send_button
        ])

    def on_suggestion_select(self, change):
        if change['new'] != 'Select a suggestion':
            self.chat_input.value = change['new']
            self.suggestions_dropdown.value = 'Select a suggestion'  # Reset the dropdown after selection

    def on_send_click(self, b):
        message = self.chat_input.value
        self.display_message(message, sender='user')

        # Here, you can add logic to process the message and generate a response
        response = process_input(message)
        self.display_message(response, sender='bot')

        self.chat_input.value = ''  # Clear the input field

    def display_message(self, message, sender):
        # Determine the side based on the sender
        if sender == 'user':
            align = 'right'
            bubble_color = '#DCF8C6'  # Light green for user
        else:
            align = 'left'
            bubble_color = '#ECECEC'  # Light gray for bot

        message_html = widgets.HTML(
            value=f"""
            <div style="display: flex; justify-content: {align}; margin: 5px;">
                <div style="background-color: {bubble_color}; padding: 10px; border-radius: 10px; max-width: 60%; word-wrap: break-word; font-size: 1rem;">
                    {message}
                </div>
            </div>
            """
        )

        self.chat_output.children += (message_html,)

    def show(self):
        self.container.layout.display = 'flex'

    def hide(self):
        self.container.layout.display = 'none'


# Collaborations Page class

In [14]:


class CollaborationsPage(PageBase):
    def create_widgets(self):
        # Create a container for the widgets
        self.container = widgets.VBox()

       # Extract unique document names
        unique_documents = list(set(doc['Document'] for doc in data))

        # Dropdown menu to choose document
        self.document_dropdown = widgets.Dropdown(
            options=unique_documents,
            description='Document:',
            disabled=False,
            layout=widgets.Layout(margin='20px 10px')
        )


        # Output area for the stacked bar graph
        self.graph_output = widgets.Output()

        # Arrange widgets in the container
        self.container.children = [self.document_dropdown, self.graph_output]

        # Event handling for dropdown change
        self.document_dropdown.observe(self.on_document_change, names='value')

    def on_document_change(self, change):
        selected_document = change['new']
        self.update_graph(selected_document)

    def update_graph(self, document):

        # Filter data for the selected document
        filtered_data = [entry for entry in data if entry['Document'] == document]

        # Prepare data for the stacked bar graph
        contributions = defaultdict(lambda: defaultdict(int))
        for entry in filtered_data:
            contributions[entry['Tab']][entry['User']] += 1

        # Convert the data to a DataFrame for easy plotting
        df1 = pd.DataFrame(contributions).fillna(0)

        # Normalize the data to show percentages
        df1 = df1.div(df1.sum(axis=0), axis=1) * 100

        with self.graph_output:
            self.graph_output.clear_output()  # Clear previous output

            # Plot the stacked bar graph
            df1.T.plot(kind='bar', stacked=True, figsize=(10, 6))

            plt.title(f"Student Contributions to {document}")
            plt.xlabel("Tabs")
            plt.ylabel("Number of Contributions")
            plt.legend(title="Students")
            plt.xticks(rotation=45)
            plt.show()


# **Main App Wrapper**

In [15]:

# ---------------------------------------- Main Wrapper ------------------------------------------



class DashboardWidget:
    def __init__(self):
        self.create_widgets()
        self.layout_widgets()
        self.setup_navigation()

        # Custom CSS
        custom_css = """
        <style>
            .custom-toggle-buttons .widget-toggle-button {
                background-color: #f7fafc !important;
                font-size: 1rem !important;  /* Adjust font size as needed */
                padding: 0.5rem 1rem 1rem 1rem;  /* Top, Right, Bottom, Left padding */
                line-height: 1;  /* Adjust line height for better text alignment */
                height: 2rem;  /* Adjust height as needed */
            }
        </style>
        """

        # Inject custom CSS into the notebook
        display(widgets.HTML(custom_css))

    def create_widgets(self):


        # Create ToggleButtons with custom class
        self.nav_items = widgets.ToggleButtons(
            options=['Main', 'Projects', 'Employees', 'Notifications','Upload json','ChatBot','Collaborations'],
            description='',
            button_style='',
            layout=widgets.Layout(width='auto', height='auto'),
        )
        self.nav_items.add_class('custom-toggle-buttons')

        # Create page instances
        self.main_page = MainPage()
        self.projects_page = ProjectsPage()
        self.employees_page = EmployeesPage()
        self.notifications_page = NotificationsPage()
        self.uploadjson_page = UploadJsonPage()
        self.chatbot_page = ChatBotPage()
        self.collaborations_page = CollaborationsPage()


    def create_header(self):
      global cur_file_name
      file_title = cur_file_name
      header_label = widgets.HTML(f"""

        <div style="display: flex; background-color: #f7fafc; padding: 1rem;">
           <div style=" display: flex; align-items: center; flex: 1;">
            <img src="https://play-lh.googleusercontent.com/yAS9WJJnjlCx77RxIvJSssrixhCdUxnBlM3CuPnQpl8QI3Ez19KreBL4xREc1gtmK_Y=w240-h480" style="height: 3rem; width: 3rem;">
            <h1 style="color:black;font-size: 1.5rem; font-family: sans-serif; font-weight: bold; margin-top: 0.5rem; margin-left: 0.5rem;">OnShape Insights - {file_title}</h1>
            </div>
            <div style="color:black;display: flex; align-items: center; justify-content: flex-end; flex: 1;">
              <b>Hello, Adam 👤</b></div>
        </div>
        """)
      return header_label

    def layout_widgets(self):
        top_bar = widgets.HBox([self.nav_items])

        main_content = widgets.VBox([
            self.main_page.container,
            self.projects_page.container,
            self.employees_page.container,
            self.notifications_page.container,
            self.uploadjson_page.container,
            self.chatbot_page.container,
            self.collaborations_page.container
        ])

        self.main_layout = widgets.VBox([self.create_header(),top_bar, main_content])
        self.main_layout.layout.width = '100%'
        self.main_layout.layout.border = '1px solid #ddd'
        self.main_layout.layout.padding = '20px'

    def setup_navigation(self):
        self.nav_items.observe(self.on_nav_change, names='value')

    def on_nav_change(self, change):
        selected_page = change.new
        if selected_page == 'Main':
            self.show_page(self.main_page)
        elif selected_page == 'Projects':
            self.show_page(self.projects_page)
        elif selected_page == 'Employees':
            self.show_page(self.employees_page)
        elif selected_page == 'Notifications':
            self.show_page(self.notifications_page)
        elif  selected_page == 'Upload json':
            self.show_page(self.uploadjson_page)
        elif selected_page == 'ChatBot':
            self.show_page(self.chatbot_page)
        elif selected_page == 'Collaborations':
            self.show_page(self.collaborations_page)

    def show_page(self, page):
        # Hide all pages
        for p in [self.main_page, self.projects_page, self.employees_page, self.notifications_page,self.uploadjson_page,self.chatbot_page, self.collaborations_page]:

            p.hide()

        # Show selected page
        page.show()

    def display(self):
        display(self.main_layout)

    def refresh_dashboard(self):

        clear_output(wait=True)
        dashboard=None
        dashboard=DashboardWidget()
        dashboard.display()



# Create and display the dashboard widget
dashboard = DashboardWidget()
dashboard.display()


HTML(value='\n        <style>\n            .custom-toggle-buttons .widget-toggle-button {\n                bac…